2018/06/7
櫻岡響

# ただの1-gramの作成

# 準備

In [12]:
from functools import reduce
from sklearn.model_selection import train_test_split
from collections import Counter

from  sqlalchemy.sql.expression import func

from anakin.lang_model.lang_model import LangModel

from anakin.lang_model_file_factory import LangModelFileFactory
from anakin.db.session import database, engine_and_session
from anakin.db.model import Base, Sentence, File, LangModelFile

from anakin.util.word import Word
from anakin.util.estimate import estimate
from anakin.util.key_to_word import KeyToWord
from anakin.util.dp_matching import DPMatching

from env import LOCAL

import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn as sns

import pickle

In [2]:
KATAKANA = [
    'アイウエオ',
    'カキクケコ',
    'サシスセソ',
    'タチツテト',
    'ナニヌネノ',
    'ハヒフヘホ',
    'マミムメモ',
    'ヤユヨ',
    'ラリルレロ',
    'ワヲンー'
]

_org = 'ヴァィゥェォガギグゲゴザジズゼゾッダヂヅデドバビブベボパピプペポャュョ'
_cov = 'ウアイウエオカキクケコサシスセソツタチツテトハヒフヘホハヒフヘホヤユヨ'

CONV_TABLE = str.maketrans(_org, _cov)

In [3]:
db = database(**LOCAL)
ENGINE, Session = engine_and_session(db, echo=False)
Base.prepare(ENGINE, reflect=True)

In [4]:
def translate_only_katakana(word_contents):
    return \
        ''.join(
            [w.yomi for w in Word.from_sentence(word_contents) if not w.yomi in Word.MARK.values()]
        )\
        .translate(CONV_TABLE)

このコードでランダム抽出出来るはずだが凄く遅いので断念
```python
%%time
#重複無しでランダム抽出出来ているかは自信がないがまあ良いだろう
seed = 1234

c = Counter()

session = Session()
query = session.query(Sentence)\
    .order_by(func.rand(seed))\
    .limit(5)

for s in query:
    only_kana = translate_only_katakana(s.contents)
    c += Counter(only_kana)
print(c)
```

In [19]:
%%time
seed = 1234

c = Counter()

session = Session()
query = session.query(Sentence).limit(100000)

for s in query:
    only_kana = translate_only_katakana(s.contents)
    c += Counter(only_kana)
print(c)

Counter({'シ': 186710, 'テ': 182969, 'イ': 182732, 'カ': 165422, 'タ': 159942, 'ン': 132822, 'ウ': 119407, 'ト': 117016, 'ノ': 101173, 'ツ': 99058, 'ク': 95592, 'マ': 90962, 'ス': 89007, 'ヨ': 85330, 'ハ': 84515, 'ニ': 73467, 'キ': 71527, 'モ': 63068, 'コ': 58763, 'ナ': 56960, 'リ': 56601, 'ル': 54924, 'オ': 49597, 'チ': 41428, 'ヤ': 40384, 'ア': 36883, 'サ': 36414, 'ラ': 35756, 'セ': 35595, 'レ': 32002, 'ホ': 31329, 'ヒ': 29810, 'ヘ': 27954, 'フ': 27699, 'ー': 26869, 'ユ': 26155, 'エ': 25761, 'ケ': 22405, 'ロ': 21726, 'ヲ': 21565, 'ソ': 20074, 'ワ': 18814, 'メ': 14670, 'ミ': 14116, 'ネ': 13972, 'ム': 6815, 'ヌ': 509})
CPU times: user 10.2 s, sys: 257 ms, total: 10.5 s
Wall time: 10.6 s


In [26]:
#ゼロ頻度問題のチェック
check = set(''.join(KATAKANA)) - set(c)
if len(check) == 0:
    print('頻度ゼロなし')
else:
    print('頻度ゼロあり')

頻度ゼロなし


In [28]:
with open('count_100000sentence.pickle', 'wb') as f:
    pickle.dump(c, f)